In [16]:
# imports
import requests
import pandas as pd
import os
import numpy as np

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [34]:
FOURSQUARE_KEY = os.getenv('FOURSQUARE_KEY')

In [35]:
bike_stations = pd.read_csv('Bike_Stations')

In [36]:
def foursquare_restaurants(latitude,longitude):
    lat = str(latitude)
    long = str(longitude)
    url= 'https://api.foursquare.com/v3/places/search'
    radius_input = 1000
    categories = '13065,13003'                                                          ##foursquare API functions differ by using different categories paramaters (13065/13003 for restaurants/19046 for transit)
    api_key = FOURSQUARE_KEY

    params_dict = {
        'radius': radius_input,
        'll': lat+','+long, 
        'categories': categories,
        'limit': 50,
        'fields': 'name,categories,distance'}
    
    header_dict = {
        "accept": "application/json",
        "Authorization": api_key}
    res = requests.get(url, params = params_dict, headers = header_dict)
    return res.json()

In [37]:
def foursquare_transit(latitude,longitude):
    lat = str(latitude)
    long = str(longitude)
    url= 'https://api.foursquare.com/v3/places/search'
    radius_input = 1000
    categories = '19046'                                                                    ##foursquare API functions differ by using different categories paramaters (13065/13003 for restaurants/19046 for transit)
    api_key = FOURSQUARE_KEY

    params_dict = {
        'radius': radius_input,
        'll': lat+','+long, 
        'categories': categories,
        'limit': 50,
        'fields': 'name,categories,distance'}
    
    header_dict = {
        "accept": "application/json",
        "Authorization": api_key}
    res = requests.get(url, params = params_dict, headers = header_dict)
    return res.json()

In [38]:
restauarant_results=[]
restaurant_distance = []                                                                                        
for _, row in bike_stations.iterrows():                                                                         ##loop through bikestations, will use row lat/long for API function inputs   
    foursquarejson = foursquare_restaurants(row['latitude'],row['longitude'])                                   ##temporarily store API function result(json)  
    restauarant_results.append(foursquarejson['results'])                                                       ##keep only the results dictionary for future reference
    restaurant_distance.append(np.min([result['distance'] for result in foursquarejson['results']]))            ##from sublists of temp variable, loop to find distance values for each restaurant, only keep shortest distance



In [40]:
station_results = []
station_distance = []                                                                                           
for _, row in bike_stations.iterrows():                                                                     ##loop through bikestations, will use row lat/long for API function inputs
    foursquarejson = foursquare_transit(row['latitude'],row['longitude'])                                   ##temporarily store API function result(json)
    station_results.append(foursquarejson['results'])                                                       ##keep only the results dictionary for future reference
    distance_lists = [result['distance'] for result in foursquarejson['results']]                           ##from sublists of temp variable, loop to find distance values for each restaurant
    if len(distance_lists)==0:                                                                              ##if a distance list is empty, return 0 because there was no train station nearby
        station_distance.append(0)
    else:
        station_distance.append(np.min(distance_lists))                                                     ## otherwise only keep shortest distance

In [41]:
len(station_distance)

164

In [42]:
len(bike_stations)

164

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [ ]:
## refer to the function comments for parsing(end goal was to find shortest distance to the selected POI)

Put your parsed results into a DataFrame

In [43]:
FourSquare_Lists = {
    'Restuarant Distance': restaurant_distance,
    'Station Distance': station_distance}

In [44]:
FourSquare_df = pd.DataFrame(FourSquare_Lists)

In [45]:
FourSquare_df

,Restuarant Distance,Station Distance
0,80,0
1,80,0
2,238,0
3,239,0
4,44,0
...,...,...
159,121,0
160,196,0
161,116,181
162,233,0


In [76]:
FourSquare_df.to_csv('FourSquare.csv', index=False)

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [56]:
YELP_KEY = os.getenv('YELP_KEY')

In [181]:
def yelp_restaurants(latitude,longitude):                                                               ##Function is called during loop, after next markdown cell
    url='https://api.yelp.com/v3/businesses/search'
    radius_input = 1000
    categories = 'restaurant,bar'
    sort_by = 'rating'
    api_key = 'Bearer '+YELP_KEY

    params_dict = {'radius':radius_input,
                   'categories': categories,
                   'latitude': latitude,
                   'longitude': longitude,
                   'sort_by': sort_by,
                   'limit': 0}
    
    header_dict = {"Authorization": api_key,
                   "accept": "application/json"}
    res = requests.get(url, params = params_dict, headers = header_dict)
    return res.json()

In [62]:
bike_stations

,latitude,longitude,available_bikes,empty_slots,name
0,25.729275,-80.241581,5,11,000605 - Virginia St & Florida Ave
1,25.732522,-80.233244,7,9,000606 - S Bayshor Dr & Aviation Ave
2,25.751100,-80.202641,5,9,000612 - SE Brickell Ave & 25th Rd
3,25.760082,-80.196217,7,9,000616 - 150 SW 15th Road
4,25.761884,-80.193101,10,4,000617 - S Miami Ave & 13th Street
...,...,...,...,...,...
159,25.785199,-80.140549,0,15,000145 - 14th & Alton
160,25.797583,-80.132518,3,13,000199 - Washington Ave & Dade Blvd
161,25.776130,-80.194220,6,0,000674 - 48 NW 2nd Street
162,25.776620,-80.141910,15,9,000130 - West Ave 6-7 St (Canopy Park)


In [94]:
test = yelp_restaurants(latitude=25.792605,longitude=-80.241581)                                        ##Ignore Testing

In [95]:
ratings = [entry['rating'] for entry in test['businesses']]                                             ##Ignore Testing

In [96]:
distance = [entry['distance'] for entry in test['businesses']]                                          ##Ignore Testing

In [97]:
names = [entry['name'] for entry in test['businesses']]                                                 ##Ignore Testing

In [112]:
station= [1 for entry in test['businesses']]                                                            ##Ignore Testing

In [106]:
parsed_dict = {'name':names,'ratings':ratings,'distance':distance}                                      ##Ignore Testing

In [108]:
print(len(ratings))                                                                                     ##Ignore Testing
print(len(distance))
print(len(names))

20
20
20


In [110]:
pd.DataFrame(parsed_dict, columns = ['name','ratings','distance'])                                      ##Ignore Testing

,name,ratings,distance
0,Miami Skate Academy,5.0,1054.502627
1,Tacos El Paso,4.5,1120.433021
2,El Zapo Enamorado,5.0,306.645646
3,La Gran Parada,4.5,715.013628
4,Galiano Cafeteria,5.0,795.186362
5,Crazy Shark,5.0,679.744906
6,SusiLicious Bakeshop,5.0,660.659445
7,El Palacio De Los Jugos,4.0,308.678178
8,Curtis Park,4.0,475.851684
9,Taco Crazy,4.5,507.450776


Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [143]:
count = 0
yelp_index = []
yelp_rating=[]
yelp_distance = []
yelp_name = []                                                                                        ##empty lists/counter
for _, row in bike_stations.iterrows():                                                               ##loop through bikestations, will use row lat/long for API function inputs  
    result = yelp_restaurants(row['latitude'],row['longitude'])                                       ##temporarily store API function result(json)                        
    yelp_rating.append([entry['rating'] for entry in result['businesses']])                           ##extract ratings from each individual result (restaurant/bar)
    yelp_distance.append([entry['distance'] for entry in result['businesses']])                       ##extract distances from each individual result (restaurant/bar)  
    yelp_name.append([entry['name'] for entry in result['businesses']])                               ##extract names from each individual result (restaurant/bar)  
    yelp_index.append([count for entry in result['businesses']])                                      ##associate an index value for each iteration which will directly correspond to the bike station   
    count = count + 1                                                                                 ##update counter   

In [142]:
bike_stations

,latitude,longitude,available_bikes,empty_slots,name
0,25.729275,-80.241581,5,11,000605 - Virginia St & Florida Ave
1,25.732522,-80.233244,7,9,000606 - S Bayshor Dr & Aviation Ave
2,25.751100,-80.202641,5,9,000612 - SE Brickell Ave & 25th Rd
3,25.760082,-80.196217,7,9,000616 - 150 SW 15th Road
4,25.761884,-80.193101,10,4,000617 - S Miami Ave & 13th Street
...,...,...,...,...,...
159,25.785199,-80.140549,0,15,000145 - 14th & Alton
160,25.797583,-80.132518,3,13,000199 - Washington Ave & Dade Blvd
161,25.776130,-80.194220,6,0,000674 - 48 NW 2nd Street
162,25.776620,-80.141910,15,9,000130 - West Ave 6-7 St (Canopy Park)


In [163]:
len(yelp_index)

164

In [170]:
yelp_index_adj = []
for value in yelp_index:
    for v in value:
        yelp_index_adj.append(v)
print(len(yelp_index_adj))

3277


In [171]:
yelp_rating_adj = []
for value in yelp_rating:
    for v in value:
        yelp_rating_adj.append(v)
print(len(yelp_rating_adj))


3277


In [173]:
yelp_distance_adj = []
for value in yelp_distance:
    for v in value:
        yelp_distance_adj.append(v)
print(len(yelp_distance_adj))

3277


In [174]:
yelp_name_adj = []
for value in yelp_name:
    for v in value:
        yelp_name_adj.append(v)
print(len(yelp_name_adj))

3277


Put your parsed results into a DataFrame

In [175]:
yelp_dict = {'yelp_index':yelp_index_adj,'yelp_name':yelp_name_adj,'yelp_rating':yelp_rating_adj,'yelp_distance':yelp_distance_adj}

In [178]:
yelp_df=pd.DataFrame(yelp_dict, columns=['yelp_index','yelp_name','yelp_rating','yelp_distance'])

In [179]:
yelp_df

,yelp_index,yelp_name,yelp_rating,yelp_distance
0,0,Farm To Fork Meals,5.0,9661.266080
1,0,Dan the Cookie Man,5.0,45344.050064
2,0,Almotti,5.0,20707.902271
3,0,Fye Stars,5.0,32224.968396
4,0,Fookem's Fabulous Key Lime Pie,5.0,173.754030
...,...,...,...,...
3272,163,Suite Habana Cafe,5.0,1044.067644
3273,163,George's Shawarma,5.0,16224.569428
3274,163,Museum of Graffiti,5.0,1201.923769
3275,163,Jenny with the pot,5.0,873.541580


In [180]:
yelp_df.to_csv('yelp_df.csv',index = False)

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

The yelp API provided more complete data because if the FourSquare API could not find the requested POI type within the radius, an empty list would be provided. As a result, additional code had to be used to replace empty values. However when the YELP API encountered the same issues, the radius parameter description confirmed that their API will provide data that may exceed the radius in order to avoid empty lists.   


Get the top 10 restaurants according to their rating

In [189]:
yelp_df.sort_values(['yelp_rating','yelp_distance'],ascending=[False,True]).head(10)            ##sort by highest rating, then lowest distance, top 10.

,yelp_index,yelp_name,yelp_rating,yelp_distance
3202,160,Rum Room,5.0,20.431553
3236,161,City Cigar Lounge,5.0,28.004793
616,30,Cielito Artisan Pops,5.0,30.871912
3048,152,Organic Food Kings,5.0,34.965540
2905,145,Acai Brazil Kosher - Miami Beach,5.0,37.402422
1343,67,Zeybek Mediterranean Cuisine,5.0,47.614079
205,10,Naranja Pina Gelato,5.0,52.253323
3097,155,Farm To Fork Meals,5.0,54.398672
2820,141,Silverlake Bistro,5.0,55.380937
388,19,Madda’s Place,5.0,56.229642
